# Code 2 - SQL Agents

## Referances
- Parsing Errors - https://github.com/langchain-ai/langchain/issues/14947

## Step 1 - Connect to Database

### Import Packages

In [1]:
from sqlite3 import connect
from langchain_community.utilities.sql_database import SQLDatabase

### Create database connection for LLM (create db)

In [2]:
db = SQLDatabase.from_uri("sqlite:///chinook.db", sample_rows_in_table_info = 3)
print(len(db.get_usable_table_names()), db.get_usable_table_names())

11 ['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


### Check Connection and Query
- The LLM Should be able to provide this output

In [3]:
con = connect("chinook.db")
cur = con.execute("SELECT Country, SUM(Total) as TotalSpent FROM Customer JOIN Invoice ON Customer.CustomerId = Invoice.CustomerId GROUP BY Country ORDER BY TotalSpent DESC LIMIT 10")
print(cur.fetchall())

[('USA', 523.06), ('Canada', 303.96), ('France', 195.1), ('Brazil', 190.1), ('Germany', 156.48), ('United Kingdom', 112.86), ('Czech Republic', 90.24), ('Portugal', 77.24), ('India', 75.26), ('Chile', 46.62)]


## Step 2 - Setup Langchain SQL Chain

### Import API Key

In [4]:
#import os
#from dotenv import load_dotenv
#load_dotenv()

In [5]:
#API_KEY = os.getenv("OPENAI_API_KEY")

### Import Packages

In [6]:
from langchain_ollama.llms import OllamaLLM

### Create an Instance of LLM with configuration & Check (creating llm)

In [7]:
llm = OllamaLLM(model="llama3.1")
llm

OllamaLLM(model='llama3.1')

## Step 3 Setup Agent

Advantages of SQL Agents:

1. Saves tokens - Retrieving only required schema & relevant tables.
2. Recover from errors by generating queries, catch the traceback and regenerate correctly
3. Chains run on a predefined pattern; Agents is defined by llm; The LLM then chooses the right tools to execute

### Import Packages

In [8]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain.agents import AgentType

### Create an Instance of Agent with llm and db

In [9]:
agent_executor = create_sql_agent(
    llm, 
    db=db, 
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True, 
    stream_runnable=False,
    agent_executor_kwargs=dict(handle_parsing_errors=True)
)

### Execute LLM Agent

In [10]:
result = agent_executor.invoke("Which country's customers spent the most?")
print(result)



> Entering new SQL Agent Executor chain...
Thought: I should look at the tables in the database to see what I can query.

Action: sql_db_list_tables
Action Input:Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, TrackI'll start by listing all the available tables in the database.

Question: Which country's customers spent the most?
Thought: I should look at the tables in the database to see what I can query.  Then I should query the schema of the most relevant tables.
Action: sql_db_list_tables
Action InputInvalid Format: Missing 'Action Input:' after 'Action:'It seems there was an error in my previous response.

To correct this, I will re-post the corrected sequence:

Question: Which country's customers spent the most?
Thought: I should look at the tables in the database to see what I can query.  Then I should query the schema of the most relevant tables.
Action: sql_db_list_tables
Action InputInvalid Format: Missing 'Action Input:' 

### Check Execution Flow

In [11]:
print(agent_executor)

name='SQL Agent Executor' verbose=True agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], partial_variables={'tools': "sql_db_query - Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.\nsql_db_schema - Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3\nsql_db_list_tables - Input is an empty string, output is a comma-separated list of tables in the database.\n

# END - Next Topic React SQL Agents